In [2]:

import os
os.environ["PYOPENGL_PLATFORM"] = "egl"
import cv2
from src.facescape_fitter import facescape_fitter
from src.utility import show_img_arr
from src.renderer import render_orthcam

fs_fitter = facescape_fitter(fs_file = "./bilinear_model_v1.6/facescape_bm_v1.6_847_50_52_id_front_new.npz", 
                             kp2d_backend = 'dlib') 
src_img = cv2.imread("./test_data/chan.jpg")  
print("reading img file start")
kp2d = fs_fitter.detect_kp2d(src_img) # extract 2D key points
mesh, a = fs_fitter.fit_kp2d(kp2d) # fit model
print("program mesh ended")

# ========== Visualize Result ==========
# transform to orthogonal camera coordinate

mesh_tm = trimesh.Trimesh(vertices = mesh.vertices.copy(), 
                          faces = fs_fitter.fv_indices_front-1, 
                          process = False)
mesh_tm.vertices[:, :2] = mesh_tm.vertices[:, 0:2] - np.array([src_img.shape[1] / 2, src_img.shape[0] / 2])#原行两列
mesh_tm.vertices = mesh_tm.vertices / src_img.shape[0] * 2
mesh_tm.vertices[:, 2] = mesh_tm.vertices[:, 2] - 10

# render texture image and depth
rend_depth, rend_tex = render_orthcam(mesh_tm, (1, 1), 
                                      rend_size = tuple(src_img.shape[:2]), 
                                      flat_shading=False)
                                      
mask = np.stack((rend_depth!=0, )*3, 2)
res_img = src_img.copy()
res_img[mask] = rend_tex[mask]

print("program ended")


# visualize
vis_scale = 512. / np.max(src_img.shape[:2])
sc_img = cv2.resize(src_img, (round(src_img.shape[1]*vis_scale), 
                              round(src_img.shape[0]*vis_scale)))

sc_res_img = cv2.resize(res_img, (round(src_img.shape[1]*vis_scale), 
                                  round(src_img.shape[0]*vis_scale)))

haha = show_img_arr(np.concatenate((sc_img, sc_res_img), 1), bgr_mode = True)
print("show succeed")






reading img file start
round 0
object function value: [907.97015]
iteration time: 0.0004799365997314453
total time: 0.0004813671112060547
round 1
GCU time: 0.033957481384277344
LSU time: 0.000751495361328125
alpha: [1.77177265e-05]
object function value: [746.98635988]
iteration time: 0.06954717636108398
total time: 0.07079029083251953
round 2
SDU time: 0.0008285045623779297
LSU time: 0.0008628368377685547
alpha: [3.2206757e-09]
object function value: [732.63753994]
iteration time: 0.041092872619628906
total time: 0.11192750930786133
round 3
SDU time: 0.0008473396301269531
LSU time: 0.0007190704345703125
alpha: [1.6843438e-05]
object function value: [501.53597776]
iteration time: 0.026814937591552734
total time: 0.1387801170349121
round 4
SDU time: 0.00099945068359375
LSU time: 0.0006575584411621094
alpha: [6.792047e-05]
object function value: [436.32073905]
iteration time: 0.02265453338623047
total time: 0.16146564483642578
round 5
SDU time: 0.0012049674987792969
LSU time: 0.000619411

EGLError: EGLError(
	err = EGL_BAD_DISPLAY,
	baseOperation = eglInitialize,
	cArguments = (
		<OpenGL._opaque.EGLDisplay_pointer object at 0x7effa063d8d0>,
		c_long(0),
		c_long(0),
	),
	result = 0
)